In [ ]:
# necessary libraries
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras import layers
from keras import models
from keras import optimizers

from keras.models import Model, Sequential
from keras import layers

import tensorflow as tf

In [ ]:
# no need to execute this for me , i already execute it and i have folders of train and valitons sets
import os
import shutil
import pandas as pd
import random

# /kaggle/input/butterfly-image-classification/train/Image_4378.jpg'

# Read the CSV file (assuming the columns are named "filename" and "label" by default)
csv_file_path = "C:/Users/MaestroCom/CNN/Training_set.csv"
data = pd.read_csv(csv_file_path)

# Creating train and validation directories
root_directory = "C:/Users/MaestroCom/CNN"  # Replace with your root directory name

image_dir="C:/Users/MaestroCom/CNN/train" 

train_directory = os.path.join(root_directory, "train")
val_directory = os.path.join(root_directory, "validation")

os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)

# Creating label directories
labels = data["label"].value_counts().index


for label in labels:
    label_train_directory = os.path.join(train_directory, label)
    label_val_directory = os.path.join(val_directory, label)
    os.makedirs(label_train_directory, exist_ok=True)
    os.makedirs(label_val_directory, exist_ok=True)

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)
# Split the data into 70% train and 30% validation
val_data_count = int(len(data) * 0.3)
validation_data = data[:val_data_count]
train_data = data[val_data_count:]


# Copy train data
for index, row in train_data.iterrows():

    # file name 
    file_name = row['filename']
    # label name
    label = row["label"]

    # images folder , all images are in here  
    source_path = os.path.join(image_dir,file_name) #
    # new destination
    destination_directory = os.path.join(train_directory, label)
    # copy files
    shutil.copy(source_path, destination_directory)
   
    
# Copy validation data
for index, row in validation_data.iterrows():
 
    file_name = row['filename']
    label = row['label']
    
    source_path = os.path.join(image_dir, file_name)
    destination_directory = os.path.join(val_directory, label)
    shutil.copy(source_path, destination_directory)

In [ ]:
#C:/Users/MaestroCom/proyek kelas work
#os.listdir("C:/Users/MaestroCom/proyek kelas work/train/POPINJAY")[:5]
os.listdir("C:/Users/MaestroCom/CNN/train/POPINJAY")[:5]

In [ ]:
os.listdir("C:/Users/MaestroCom/CNN/validation/POPINJAY")[:5]

In [ ]:
#directories
training_dir="C:/Users/MaestroCom/CNN/train"
validation_dir="C:/Users/MaestroCom/CNN/validation"

In [ ]:
import os

# directories
train_dir = training_dir
validation_dir = validation_dir

# calculate distribution of classes for train
train_class_counts = {}
for class_folder in os.listdir(train_dir):
    class_path = os.path.join(train_dir, class_folder)
    if os.path.isdir(class_path):
        num_images = len(os.listdir(class_path))
        train_class_counts[class_folder] = num_images

# calculate distribution of classes for valdiation
validation_class_counts = {}
for class_folder in os.listdir(validation_dir):
    class_path = os.path.join(validation_dir, class_folder)
    if os.path.isdir(class_path):
        num_images = len(os.listdir(class_path))
        validation_class_counts[class_folder] = num_images

print("Training set Distribution:")
print(train_class_counts)

print("Validation set Distribution:")
print(validation_class_counts)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.bar(train_class_counts.keys(), train_class_counts.values())
plt.title('Training set Distribution')
plt.xlabel('Classes')
plt.ylabel('Sample Numbers')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.bar(validation_class_counts.keys(), validation_class_counts.values())
plt.title('Validation set Distribution')
plt.xlabel('Classes')
plt.ylabel('Sample Numbers')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# use this function for preparing data  
def prep_data(augmented,batch_size=16):      # if you want to augmented dat set use it like this : prep_data(True)
    if augmented:                            # default batch_size is 16 , you can change it 
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)
    
        validation_datagen = ImageDataGenerator(rescale=1./255)    

    else:
        train_datagen = ImageDataGenerator(rescale=1.0 / 255.0)
        validation_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

    # training set
    train_set = train_datagen.flow_from_directory(
        training_dir,
        target_size=(180, 180),  # The dimensions to which all images found will be resized
        batch_size=batch_size,# 32  default
        class_mode="sparse") # you can change this to onehotEncoded format or another format
         
    
    # validation set
    validation_set = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(180, 180),
        batch_size=batch_size,  # 32 default
        class_mode="sparse")
             
    return train_set , validation_set

In [ ]:
# visulization function for Models
def visualize(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(acc) + 1)
    
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    
    axs[0].plot(epochs, acc, 'r', label='Training acc')
    axs[0].plot(epochs, val_acc, 'b', label='Validation acc')
    axs[0].set_title('Training and validation accuracy')
    axs[0].legend()
    axs[0].grid(True)
    
    axs[1].plot(epochs, loss, 'r', label='Training loss')
    axs[1].plot(epochs, val_loss, 'b', label='Validation loss')
    axs[1].set_title('Training and validation loss')
    axs[1].legend()
    axs[1].grid(True)
    
    plt.tight_layout()
    plt.show()

In [ ]:
# not augmented dataset
#straight_train_set,validation_set=prep_data(False)

#images,labels=straight_train_set.next()

#class_names = straight_train_set.class_indices
#class_names = {v: k for k, v in class_names.items()}

In [ ]:
# not augmented dataset
straight_train_set, validation_set = prep_data(False)

# Iterate over the batches of images and labels
for images, labels in straight_train_set:
    # Process the batch
    # ...
    break  # exit the loop after the first batch

class_names = straight_train_set.class_indices
class_names = {v: k for k, v in class_names.items()}

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))

for i in range(4):
    axes[i].imshow(images[i]) 
    label_index = int(labels[i])
    class_name = class_names[label_index]
    axes[i].set_title(f"{class_name}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# augmented dataset
#augmented_train_set,validation_set=prep_data(True)
#images,labels=augmented_train_set.next()

#class_names = augmented_train_set.class_indices
#class_names = {v: k for k, v in class_names.items()}

In [ ]:
# augmented dataset
augmented_train_set, validation_set = prep_data(True)

# Get the batch size
batch_size = augmented_train_set.batch_size

# Iterate over the batches of images and labels
for i in range(augmented_train_set.samples // batch_size):
    images, labels = next(iter(augmented_train_set))
    # Process the batch
    # ...
    break  # exit the loop after the first batch

class_names = augmented_train_set.class_indices
class_names = {v: k for k, v in class_names.items()}

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(15, 5))

for i in range(4):
    axes[i].imshow(images[i]) 
    label_index = int(labels[i])
    class_name = class_names[label_index]
    axes[i].set_title(f"{class_name}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

#### cnn model 1_!

In [ ]:
train_set,validation_set=prep_data(True)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(75 , activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
#import tensorflow as tf

# Define the model architecture
#model = tf.keras.models.Sequential([
    # Add layers to the model
    # ...
#])

# Compile the model
#model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-4),
              #loss='sparse_categorical_crossentropy',
              #metrics=['accuracy'])

In [ ]:
model1_1 = model.fit(
    train_set,
    epochs=30,
    validation_data=validation_set,)

In [ ]:
visualize(model1_1)

#### CNN MODEL 1_2

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(75 , activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=2e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model1_1 = model.fit(
    train_set,
    epochs=50,
    validation_data=validation_set,)

In [ ]:
visualize(model1_1)

#### Model 1_4

In [ ]:
train_set,validation_set=prep_data(True)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(75 , activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), # 0.0002
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model1_3 = model.fit(
    train_set,
    epochs=100,
    validation_data=validation_set,)

In [ ]:
visualize(model1_3)

#### Model 1_5

In [ ]:
train_set,validation_set=prep_data(True)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(75 , activation='softmax'))

model.summary()


In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), # 0.0002
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model1_5 = model.fit(
    train_set,
    epochs=120,
    validation_data=validation_set,)

In [ ]:
visualize(model1_5)

In [ ]:
#### Model 1_6

In [ ]:
train_set,validation_set=prep_data(True)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.45))
model.add(layers.Dense(75 , activation='softmax'))

model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001), # 0.0002
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model1_5 = model.fit(
    train_set,
    epochs=100,
    validation_data=validation_set,)

In [ ]:
visualize(model1_5)

#### Transfer Learning

In [ ]:
# importing pretrained models 
# importing pretrained models 
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import Xception

#### Inception V3 Fine Tuning

In [ ]:
train_set,validation_set=prep_data(True,batch_size=16) # with Data Augmentation

In [ ]:
base_model = InceptionV3(weights='imagenet',
    include_top=False,
    input_shape=(180, 180, 3))

In [ ]:
# Freeze the layers except the last few layers
for layer in base_model.layers[:-15]:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(75, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005), 
    metrics=['accuracy']
)

model.summary()

In [ ]:
fine_tune_inception1_1 = model.fit(
    train_set,
    epochs=40,
    validation_data=validation_set,
    )

In [ ]:
visualize(fine_tune_inception1_1)

#### Inception V3 Fine Tuning 1_2

In [ ]:
train_set,validation_set=prep_data(True,batch_size=16) # with Data Augmentation

In [ ]:
base_model = InceptionV3(weights='imagenet',
    include_top=False,
    input_shape=(180, 180, 3))

In [ ]:
# Freeze the layers except the last few layers
for layer in base_model.layers[:-18]:
    layer.trainable = False

In [ ]:
model = Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(90, activation='softmax'))

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00005),
    metrics=['accuracy']
)

model.summary()

In [ ]:
fine_tune_inception1_2 = model.fit(
    train_set,
    epochs=30,
    validation_data=validation_set,
    )

In [ ]:
visualize(fine_tune_inception1_2)